In [1]:
import os
import sys

home=os.path.expanduser("~")
os.environ["SPARK_HOME"]=os.path.join(home, 'spark-2.0.0-bin-hadoop2.7')
os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.1-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

In [2]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder.master("local").appName("myApp").config(conf=myConf).getOrCreate()

# TF-IDF(Term Frequency-Inverse Document Frequency)

* tf	term frequency 단어의 빈도 수	ft,d / (number of words in d) = 1/4 = 0.25
(3번째 문서에 stopwords를 제외하면 4개의 단어, wisdom은 1회 나타난다.)
* df	document frequency 단어가 나타난 문서 수	3 (wisdom이 포함된 문서는 3)
* N	number of documents 전체 문서의 수	11 (전체의 문서는 11개)
* idf	inverse document frequency 단어가 나타난 문서의 비율을 거꾸로	ln(N+1 / df+1) + 1 = log(12/4) + 1 = 1.09861 + 1
0으로 나뉘는 것을 방지하기 위해 smoothing, 즉 1을 더한다.

In [4]:
doc=[
    "When I find myself in times of trouble",
    "Mother Mary comes to me",
    "Speaking words of wisdom, let it be",
    "And in my hour of darkness",
    "She is standing right in front of me",
    "Speaking words of wisdom, let it be",
    "Let it be",
    "Let it be",
    "Let it be",
    "Let it be",
    "Whisper words of wisdom, let it be"
]
import math
tf=1./4
df=3.
N=11.
idf=math.log((N+1)/(df+1))+1
print idf

2.09861228867


In [13]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 
print vectorizer.fit_transform(doc).todense()
print vectorizer.vocabulary_

[[0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 1 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1]
 [1 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1]
 [0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1]]
{u'and': 0, u'be': 1, u'right': 17, u'whisper': 25, u'is': 8, u'it': 9, u'wisdom': 26, u'me': 12, u'let': 10, u'words': 27, u'in': 7, u'front': 5, u'trouble': 23, u'find': 4, u'standing': 20, u'comes': 2, u'myself': 15, u'darkness': 3, u'hour': 6, u'of': 16, u'when': 24, u'times': 21, u'to': 22, u'she': 18, u'mother': 13, u'my': 14, u'mary': 11, u'

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, stop_words='english',norm = None)
print vectorizer.fit_transform(doc)
print vectorizer.vocabulary_
print vectorizer.idf_

  (0, 9)	2.791759469228055
  (0, 10)	2.791759469228055
  (1, 5)	2.791759469228055
  (1, 4)	2.791759469228055
  (1, 0)	2.791759469228055
  (2, 7)	2.386294361119891
  (2, 13)	2.09861228866811
  (2, 12)	2.09861228866811
  (2, 3)	1.4054651081081644
  (3, 2)	2.791759469228055
  (3, 1)	2.791759469228055
  (4, 8)	2.791759469228055
  (4, 6)	2.791759469228055
  (5, 7)	2.386294361119891
  (5, 13)	2.09861228866811
  (5, 12)	2.09861228866811
  (5, 3)	1.4054651081081644
  (6, 3)	1.4054651081081644
  (7, 3)	1.4054651081081644
  (8, 3)	1.4054651081081644
  (9, 3)	1.4054651081081644
  (10, 13)	2.09861228866811
  (10, 12)	2.09861228866811
  (10, 3)	1.4054651081081644
  (10, 11)	2.791759469228055
{u'standing': 8, u'right': 6, u'darkness': 1, u'hour': 2, u'whisper': 11, u'times': 9, u'let': 3, u'speaking': 7, u'words': 13, u'mother': 5, u'trouble': 10, u'wisdom': 12, u'mary': 4, u'comes': 0}
[2.79175947 2.79175947 2.79175947 1.40546511 2.79175947 2.79175947
 2.79175947 2.38629436 2.79175947 2.79175947 2.

In [18]:
doc=[
    ["When I find myself in times of trouble"],
    ["Mother Mary comes to me"],
    ["Speaking words of wisdom, let it be"],
    ["And in my hour of darkness"],
    ["She is standing right in front of me"],
    ["Speaking words of wisdom, let it be"],
    [u"우리 Let it be"],
    [u"나 Let it be"],
    [u"너 Let it be"],
    ["Let it be"],
    ["Whisper words of wisdom, let it be"]
]

myDf=spark.createDataFrame(doc,['sent'])
myDf.show(truncate=False)

+--------------------------------------+
|sent                                  |
+--------------------------------------+
|When I find myself in times of trouble|
|Mother Mary comes to me               |
|Speaking words of wisdom, let it be   |
|And in my hour of darkness            |
|She is standing right in front of me  |
|Speaking words of wisdom, let it be   |
|우리 Let it be                          |
|나 Let it be                           |
|너 Let it be                           |
|Let it be                             |
|Whisper words of wisdom, let it be    |
+--------------------------------------+



# StringIndexer

In [19]:
from pyspark.ml.feature import StringIndexer
labelIndexer = StringIndexer(inputCol="sent", outputCol="sentLabel")
model=labelIndexer.fit(myDf)
siDf=model.transform(myDf)
siDf.show()

+--------------------+---------+
|                sent|sentLabel|
+--------------------+---------+
|When I find mysel...|      9.0|
|Mother Mary comes...|      8.0|
|Speaking words of...|      0.0|
|And in my hour of...|      5.0|
|She is standing r...|      4.0|
|Speaking words of...|      0.0|
|        우리 Let it be|      6.0|
|         나 Let it be|      1.0|
|         너 Let it be|      2.0|
|           Let it be|      7.0|
|Whisper words of ...|      3.0|
+--------------------+---------+



# Tokenizer

In [21]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="sent", outputCol="words")
tokDf = tokenizer.transform(myDf)
for r in tokDf.select("sent", "words").take(3):
    print r

Row(sent=u'When I find myself in times of trouble', words=[u'when', u'i', u'find', u'myself', u'in', u'times', u'of', u'trouble'])
Row(sent=u'Mother Mary comes to me', words=[u'mother', u'mary', u'comes', u'to', u'me'])
Row(sent=u'Speaking words of wisdom, let it be', words=[u'speaking', u'words', u'of', u'wisdom,', u'let', u'it', u'be'])


### RegTokenizer

In [34]:
from pyspark.ml.feature import RegexTokenizer
re = RegexTokenizer(inputCol="sent", outputCol="wordsReg", pattern="\s+")
reDf=re.transform(myDf)
reDf.show()

+--------------------+--------------------+
|                sent|            wordsReg|
+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|
|Mother Mary comes...|[mother, mary, co...|
|Speaking words of...|[speaking, words,...|
|And in my hour of...|[and, in, my, hou...|
|She is standing r...|[she, is, standin...|
|Speaking words of...|[speaking, words,...|
|        우리 Let it be|   [우리, let, it, be]|
|         나 Let it be|    [나, let, it, be]|
|         너 Let it be|    [너, let, it, be]|
|           Let it be|       [let, it, be]|
|Whisper words of ...|[whisper, words, ...|
+--------------------+--------------------+



### Stopwords
* http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words

In [37]:
from pyspark.ml.feature import StopWordsRemover
stopwords=list()
_stopwords=stop.getStopWords()
for e in _stopwords:
    stopwords.append(e)

_mystopwords=[u"나",u"너", u"우리"]
for e in _mystopwords:
    stopwords.append(e)
stop.setStopWords(stopwords)

for e in stop.getStopWords():
    print e,

i me my myself we our ours ourselves you your yours yourself yourselves he him his himself she her hers herself it its itself they them their theirs themselves what which who whom this that these those am is are was were be been being have has had having do does did doing a an the and but if or because as until while of at by for with about against between into through during before after above below to from up down in out on off over under again further then once here there when where why how all any both each few more most other some such no nor not only own same so than too very s t can will just don should now d ll m o re ve y ain aren couldn didn doesn hadn hasn haven isn ma mightn mustn needn shan shouldn wasn weren won wouldn 나 너 우리 나 너 우리


In [38]:
stopDf=stop.transform(reDf)
stopDf.show()

+--------------------+--------------------+--------------------+
|                sent|            wordsReg|             nostops|
+--------------------+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|[find, times, tro...|
|Mother Mary comes...|[mother, mary, co...|[mother, mary, co...|
|Speaking words of...|[speaking, words,...|[speaking, words,...|
|And in my hour of...|[and, in, my, hou...|    [hour, darkness]|
|She is standing r...|[she, is, standin...|[standing, right,...|
|Speaking words of...|[speaking, words,...|[speaking, words,...|
|        우리 Let it be|   [우리, let, it, be]|               [let]|
|         나 Let it be|    [나, let, it, be]|               [let]|
|         너 Let it be|    [너, let, it, be]|               [let]|
|           Let it be|       [let, it, be]|               [let]|
|Whisper words of ...|[whisper, words, ...|[whisper, words, ...|
+--------------------+--------------------+--------------------+



# CountVectorizer

In [48]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="nostops", outputCol="cv", vocabSize=30,minDF=1.0)
cvModel = cv.fit(stopDf)
cvDf = cvModel.transform(stopDf)

cvDf.select('sent','nostops','cv').show()
for v in cvModel.vocabulary:
    print v,

+--------------------+--------------------+--------------------+
|                sent|             nostops|                  cv|
+--------------------+--------------------+--------------------+
|When I find mysel...|[find, times, tro...|(16,[8,9,14],[1.0...|
|Mother Mary comes...|[mother, mary, co...|(16,[11,12,13],[1...|
|Speaking words of...|[speaking, words,...|(16,[0,1,2,3],[1....|
|And in my hour of...|    [hour, darkness]|(16,[5,10],[1.0,1...|
|She is standing r...|[standing, right,...|(16,[4,6,15],[1.0...|
|Speaking words of...|[speaking, words,...|(16,[0,1,2,3],[1....|
|        우리 Let it be|               [let]|      (16,[0],[1.0])|
|         나 Let it be|               [let]|      (16,[0],[1.0])|
|         너 Let it be|               [let]|      (16,[0],[1.0])|
|           Let it be|               [let]|      (16,[0],[1.0])|
|Whisper words of ...|[whisper, words, ...|(16,[0,1,2,7],[1....|
+--------------------+--------------------+--------------------+

let wisdom, words speaki

# TF-IDF

In [49]:
from pyspark.ml.feature import HashingTF, IDF

hashTF = HashingTF(inputCol="nostops", outputCol="hash", numFeatures=50)
hashDf = hashTF.transform(stopDf)
idf = IDF(inputCol="hash", outputCol="idf")
idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)
for e in idfDf.select("nostops","hash").take(10):
    print(e)

Row(nostops=[u'find', u'times', u'trouble'], hash=SparseVector(50, {10: 1.0, 24: 1.0, 43: 1.0}))
Row(nostops=[u'mother', u'mary', u'comes'], hash=SparseVector(50, {1: 1.0, 21: 1.0, 24: 1.0}))
Row(nostops=[u'speaking', u'words', u'wisdom,', u'let'], hash=SparseVector(50, {9: 1.0, 12: 1.0, 14: 1.0, 41: 1.0}))
Row(nostops=[u'hour', u'darkness'], hash=SparseVector(50, {23: 1.0, 27: 1.0}))
Row(nostops=[u'standing', u'right', u'front'], hash=SparseVector(50, {24: 1.0, 43: 1.0, 46: 1.0}))
Row(nostops=[u'speaking', u'words', u'wisdom,', u'let'], hash=SparseVector(50, {9: 1.0, 12: 1.0, 14: 1.0, 41: 1.0}))
Row(nostops=[u'let'], hash=SparseVector(50, {14: 1.0}))
Row(nostops=[u'let'], hash=SparseVector(50, {14: 1.0}))
Row(nostops=[u'let'], hash=SparseVector(50, {14: 1.0}))
Row(nostops=[u'let'], hash=SparseVector(50, {14: 1.0}))


# NGRAM

In [50]:
from pyspark.ml.feature import NGram
ngram = NGram(n=2, inputCol="words", outputCol="ngrams")
ngramDf = ngram.transform(tokDf)
ngramDf.show()
for e in ngramDf.select("words","ngrams").take(3):
    print e

+--------------------+--------------------+--------------------+
|                sent|               words|              ngrams|
+--------------------+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|[when i, i find, ...|
|Mother Mary comes...|[mother, mary, co...|[mother mary, mar...|
|Speaking words of...|[speaking, words,...|[speaking words, ...|
|And in my hour of...|[and, in, my, hou...|[and in, in my, m...|
|She is standing r...|[she, is, standin...|[she is, is stand...|
|Speaking words of...|[speaking, words,...|[speaking words, ...|
|        우리 Let it be|   [우리, let, it, be]|[우리 let, let it, ...|
|         나 Let it be|    [나, let, it, be]|[나 let, let it, i...|
|         너 Let it be|    [너, let, it, be]|[너 let, let it, i...|
|           Let it be|       [let, it, be]|     [let it, it be]|
|Whisper words of ...|[whisper, words, ...|[whisper words, w...|
+--------------------+--------------------+--------------------+

Row(words=[u'when', u'i'

# 연속데이터의 변환

In [63]:
%%writefile data/ds_spark_heightweight.txt
1   65.78   112.99
2   71.52   136.49
3   69.40   153.03
4   68.22   142.34
5   67.79   144.30
6   68.70   123.30
7   69.80   141.49
8   70.01   136.46
9   67.90   112.37
10  66.78   120.67
11  66.49   127.45
12  67.62   114.14
13  68.30   125.61
14  67.12   122.46
15  68.28   116.09
16  71.09   140.00
17  66.46   129.50
18  68.65   142.97
19  71.23   137.90
20  67.13   124.04
21  67.83   141.28
22  68.88   143.54
23  63.48   97.90
24  68.42   129.50
25  67.63   141.85
26  67.21   129.72
27  70.84   142.42
28  67.49   131.55
29  66.53   108.33
30  65.44   113.89
31  69.52   103.30
32  65.81   120.75
33  67.82   125.79
34  70.60   136.22
35  71.80   140.10
36  69.21   128.75
37  66.80   141.80
38  67.66   121.23
39  67.81   131.35
40  64.05   106.71
41  68.57   124.36
42  65.18   124.86
43  69.66   139.67
44  67.97   137.37
45  65.98   106.45
46  68.67   128.76
47  66.88   145.68
48  67.70   116.82
49  69.82   143.62
50  69.09   134.93

Overwriting data/ds_spark_heightweight.txt


In [68]:
from pyspark.sql.types import *

rdd=spark.sparkContext\
    .textFile(os.path.join('data','ds_spark_heightweight.txt'))

myRdd=rdd.map(lambda line:[float(x) for x in line.split()])
myDf=spark.createDataFrame(myRdd,["id","weight","height"])

In [69]:
myDf.printSchema()

root
 |-- id: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- height: double (nullable = true)



In [70]:
from pyspark.ml.feature import Binarizer
binarizer = Binarizer(threshold=68.0, inputCol="weight", outputCol="weight2")
binDf = binarizer.transform(myDf)
binDf.show(10)

+----+------+------+-------+
|  id|weight|height|weight2|
+----+------+------+-------+
| 1.0| 65.78|112.99|    0.0|
| 2.0| 71.52|136.49|    1.0|
| 3.0|  69.4|153.03|    1.0|
| 4.0| 68.22|142.34|    1.0|
| 5.0| 67.79| 144.3|    0.0|
| 6.0|  68.7| 123.3|    1.0|
| 7.0|  69.8|141.49|    1.0|
| 8.0| 70.01|136.46|    1.0|
| 9.0|  67.9|112.37|    0.0|
|10.0| 66.78|120.67|    0.0|
+----+------+------+-------+
only showing top 10 rows



In [71]:
from pyspark.ml.feature import QuantileDiscretizer

discretizer = QuantileDiscretizer(numBuckets=3, inputCol="height", outputCol="height3")
qdDf = discretizer.fit(binDf).transform(binDf)
qdDf.show(10)

+----+------+------+-------+-------+
|  id|weight|height|weight2|height3|
+----+------+------+-------+-------+
| 1.0| 65.78|112.99|    0.0|    0.0|
| 2.0| 71.52|136.49|    1.0|    1.0|
| 3.0|  69.4|153.03|    1.0|    2.0|
| 4.0| 68.22|142.34|    1.0|    2.0|
| 5.0| 67.79| 144.3|    0.0|    2.0|
| 6.0|  68.7| 123.3|    1.0|    0.0|
| 7.0|  69.8|141.49|    1.0|    2.0|
| 8.0| 70.01|136.46|    1.0|    1.0|
| 9.0|  67.9|112.37|    0.0|    0.0|
|10.0| 66.78|120.67|    0.0|    0.0|
+----+------+------+-------+-------+
only showing top 10 rows



In [108]:
import os
import urllib
_url = 'https://people.sc.fsu.edu/~jburkardt/data/csv/hw_200.csv'
_fname = os.path.join(os.getcwd(),'data','hw_200.csv')

if(not os.path.exists(_fname)):
    urllib.urlretrieve(_url,_fname)
    
df = spark.read.format('com.databricks.spark.csv')\
    .options(header='true', inferschema='true').load('data/hw_200.csv')

df.printSchema()

root
 |-- Index: integer (nullable = true)
 |--  Height(Inches)": double (nullable = true)
 |--  "Weight(Pounds)": double (nullable = true)



In [126]:
import requests
r=requests.get('https://people.sc.fsu.edu/~jburkardt/data/csv/hw_200.csv')
wc=r.content.splitlines()

ma = map(lambda x : x.split(','), wc)

ma

[['"Index"', ' Height(Inches)"', ' "Weight(Pounds)"'],
 ['1', ' 65.78', ' 112.99'],
 ['2', ' 71.52', ' 136.49'],
 ['3', ' 69.40', ' 153.03'],
 ['4', ' 68.22', ' 142.34'],
 ['5', ' 67.79', ' 144.30'],
 ['6', ' 68.70', ' 123.30'],
 ['7', ' 69.80', ' 141.49'],
 ['8', ' 70.01', ' 136.46'],
 ['9', ' 67.90', ' 112.37'],
 ['10', ' 66.78', ' 120.67'],
 ['11', ' 66.49', ' 127.45'],
 ['12', ' 67.62', ' 114.14'],
 ['13', ' 68.30', ' 125.61'],
 ['14', ' 67.12', ' 122.46'],
 ['15', ' 68.28', ' 116.09'],
 ['16', ' 71.09', ' 140.00'],
 ['17', ' 66.46', ' 129.50'],
 ['18', ' 68.65', ' 142.97'],
 ['19', ' 71.23', ' 137.90'],
 ['20', ' 67.13', ' 124.04'],
 ['21', ' 67.83', ' 141.28'],
 ['22', ' 68.88', ' 143.54'],
 ['23', ' 63.48', ' 97.90'],
 ['24', ' 68.42', ' 129.50'],
 ['25', ' 67.63', ' 141.85'],
 ['26', ' 67.21', ' 129.72'],
 ['27', ' 70.84', ' 142.42'],
 ['28', ' 67.49', ' 131.55'],
 ['29', ' 66.53', ' 108.33'],
 ['30', ' 65.44', ' 113.89'],
 ['31', ' 69.52', ' 103.30'],
 ['32', ' 65.81', ' 120.7

# VectorAssembler

In [72]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler(inputCols=["weight2","height3"],outputCol="features")
vaDf = va.transform(qdDf)
vaDf.printSchema()
vaDf.show(5)

root
 |-- id: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- height: double (nullable = true)
 |-- weight2: double (nullable = true)
 |-- height3: double (nullable = true)
 |-- features: vector (nullable = true)

+---+------+------+-------+-------+---------+
| id|weight|height|weight2|height3| features|
+---+------+------+-------+-------+---------+
|1.0| 65.78|112.99|    0.0|    0.0|(2,[],[])|
|2.0| 71.52|136.49|    1.0|    1.0|[1.0,1.0]|
|3.0|  69.4|153.03|    1.0|    2.0|[1.0,2.0]|
|4.0| 68.22|142.34|    1.0|    2.0|[1.0,2.0]|
|5.0| 67.79| 144.3|    0.0|    2.0|[0.0,2.0]|
+---+------+------+-------+-------+---------+
only showing top 5 rows



# pipeline

In [73]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.classification import LogisticRegression

df = spark.createDataFrame([
    (0L, "a b c d e spark", 1.0),
    (1L, "b d", 0.0),
    (2L, "spark f g h", 1.0),
    (3L, "hadoop mapreduce", 0.0),
    (4L, "my dog has flea problems. help please.",0.0)
    ], ["id", "text", "label"])

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.01)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# Fit the pipeline to training documents.
model = pipeline.fit(df)
myDf = model.transform(df)

# 예제 연설문

In [75]:
%%writefile data/20191021_policeAddress.txt
존경하는 국민 여러분, 경찰관 여러분, 일흔네 돌 ‘경찰의 날’입니다.
 
국민의 안전을 위해 밤낮없이 애쓰시는 전국의 15만 경찰관 여러분께 먼저 감사를 드립니다. 전몰·순직 경찰관들의 고귀한 희생에 경의를 표합니다. 유가족 여러분께 위로의 마음을 전합니다.
 
오늘 홍조근정훈장을 받으신 중앙경찰학교장 이은정 치안감님, 근정포장을 받으신 광주남부경찰서 김동현 경감님을 비롯한 수상자 여러분께 각별한 축하와 감사를 드립니다. 또한 경찰 영웅으로 추서되신 차일혁, 최중락님께 국민의 사랑을 전해드립니다.
 
사랑하는 경찰관 여러분,
 
여러분의 헌신적 노력으로 우리의 치안은 참 좋아졌습니다. 지난해 범죄 발생은 2015년에 비해 15.1% 줄었습니다. 같은 기간 교통사고 사망자는 18.2% 감소했습니다.
 
치안의 개선은 국민의 체감으로 나타나고 있습니다. 올해 상반기 국민의 체감안전도는 74.5점으로 역대 최고를 기록했습니다. 범죄안전도는 처음으로 80점을 넘었습니다.
 
한국을 찾는 외국 관광객들도 우리의 치안을 가장 좋게 평가합니다. 한국의 무엇이 좋았느냐는 물음에 외국 관광객들은 7년 연속으로 치안이 가장 좋았다고 응답했습니다. 개발도상국들은 우리의 경찰을 모범으로 삼으려 합니다.
 
올해는 ‘경찰의 날’에 맞춰 국제치안산업박람회와 서울국제경찰청장회의가 함께 열립니다. 우리의 치안 발전과 치안산업 발전이 세계에 더 널리 알려지게 될 것입니다.
 
자랑스러운 경찰관 여러분,
 
경찰헌장은 “나라와 겨레를 위하여 충성”을 다한다는 다짐으로 시작합니다. 헌장처럼 우리 경찰은 ‘나라와 겨레를 위한 충성’의 길을 걸으려 노력해 왔습니다.
 
대한민국 경찰은 1945년 광복 직후에 공식 탄생했습니다. 그러나 그 뿌리는 대한민국 임시정부에 닿아 있습니다.
 
임시정부 초대 경무국장 백범 김구 선생과 나석주, 나창헌, 유상근 의사 등 임시정부 경찰은 앞장서서 일제와 싸웠습니다. 일본 관헌에게 폭탄을 던지고, 밀정을 응징하며, 임정 요인들을 보호했습니다.
 
광복 이후 6‧25전쟁에서도 경찰은 국군과 함께 피를 흘렸습니다. 그 전쟁에서 1만여 명의 경찰관이 목숨을 잃었습니다. 그 후로도 경찰은 국민의 안전을 지키고 국가의 안보를 도왔습니다. 역대 경찰의 헌신에 대해 국민과 함께 거듭 감사의 말씀을 드립니다. 감사합니다.
 
그러나 잘못도 없지는 않았습니다. 한때 경찰은 공권력을 무리하게 집행하며 국민의 인권을 훼손했습니다. 부실하거나 불공정한 수사로 국민의 지탄을 받은 적도 있습니다. 무기력한 법 집행으로 국민께 걱정을 드리기도 했습니다.
 
지금 경찰은 과거를 돌아보며 국민과 국가에 충성하는 경찰로 거듭나려고 노력하고 있습니다. 경찰은 문재인 정부 들어 가장 먼저 개혁위원회를 만들고 자체개혁에 나섰습니다. 경찰의 개혁을 국민은 큰 기대로 주목하고 있습니다.
 
검경 수사권 조정과 자치경찰제 도입이 국회에서 논의되고 있습니다. 국회가 조속히 입법을 매듭지어 주시기 바랍니다. 그리하여 경찰이 중립성, 공정성, 전문성을 갖추고 본연의 임무를 충실히 수행하는 선진경찰로 더욱 발전해 주기를 소망합니다.
 
정부는 경찰의 근무여건을 개선하기 위해 노력하고 있습니다. 이미 경찰관 8,572명을 늘렸고, 앞으로도 충원을 계속할 것입니다. 특히 일선 경찰의 근무환경을 개선하겠습니다.
 
정부는 누구도 법 위에 군림하지 못하는 법치주의를 확립하고자 합니다. 그러자면 검찰과 경찰이 법을 누구에게나 엄정하고 공정하게 집행해야 됩니다. 수사 또한 엄정하고 공정해야 합니다. 동시에 검찰과 경찰 스스로도 법을 엄격히 준수해야 합니다. 특히 공권력이 인권의 제약을 수반하는 경우에는 절제하며 행사하는 것이 마땅합니다. 검찰개혁과 경찰개혁은 더 미룰 수 없는 시대적 과제가 됐습니다.
 
경찰헌장은 따뜻한 경찰, 의로운 경찰, 공정한 경찰을 다짐합니다. 흔들림 없이 그 길로 가시기 바랍니다. 국민이 여러분을 응원할 것입니다.

행사를 준비하신 민갑룡 경찰청장님과 관계자 여러분, 고맙습니다. 함께하신 진영 행정안전부 장관님, 이용범 인천시의회 의장님, 박정훈 경찰위원장님, 강영규 경우회장님과 역대 경찰청장님, 그리고 우리 시민 경찰님들을 비롯한 내빈 여러분, 고맙습니다.
 
감사합니다.

Writing data/20191021_policeAddress.txt


In [76]:
from pyspark.sql.types import StructType, StructField, StringType
police=spark.read\
    .option("header","true")\
    .option("delimiter"," ")\
    .option("inferSchema","true")\
    .schema(
        StructType([
            StructField("sent",StringType()),
            ])
    )\
    .text(os.path.join("data", "20191021_policeAddress.txt")) 


In [119]:
police.show(5)

+--------------------+
|                sent|
+--------------------+
|존경하는 국민 여러분, 경찰관 ...|
|                    |
|국민의 안전을 위해 밤낮없이 애...|
|                    |
|오늘 홍조근정훈장을 받으신 중앙...|
+--------------------+
only showing top 5 rows



In [121]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="sent", outputCol="words")
tokDf = tokenizer.transform(police)
tokDf.show(5)

+--------------------+--------------------+
|                sent|               words|
+--------------------+--------------------+
|존경하는 국민 여러분, 경찰관 ...|[존경하는, 국민, 여러분,, ...|
|                    |                  []|
|국민의 안전을 위해 밤낮없이 애...|[국민의, 안전을, 위해, 밤낮...|
|                    |                  []|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|
+--------------------+--------------------+
only showing top 5 rows



In [127]:
for r in tokDf.select("sent").take(3):
    print r[0]

존경하는 국민 여러분, 경찰관 여러분, 일흔네 돌 ‘경찰의 날’입니다.
 
국민의 안전을 위해 밤낮없이 애쓰시는 전국의 15만 경찰관 여러분께 먼저 감사를 드립니다. 전몰·순직 경찰관들의 고귀한 희생에 경의를 표합니다. 유가족 여러분께 위로의 마음을 전합니다.


In [129]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="words", outputCol="nostops")
stop.setStopWords([u"돌",u"너", u"우리"])
_mystopwords=[u"나",u"너", u"우리"]

stopwords = list()
for e in _mystopwords:
    stopwords.append(e)
stop.setStopWords(stopwords)

StopWordsRemover_4cfd80e4116081c1a529

In [132]:
stopDf=stop.transform(tokDf)
stopDf.show()

+--------------------+--------------------+--------------------+
|                sent|               words|             nostops|
+--------------------+--------------------+--------------------+
|존경하는 국민 여러분, 경찰관 ...|[존경하는, 국민, 여러분,, ...|[존경하는, 국민, 여러분,, ...|
|                    |                  []|                  []|
|국민의 안전을 위해 밤낮없이 애...|[국민의, 안전을, 위해, 밤낮...|[국민의, 안전을, 위해, 밤낮...|
|                    |                  []|                  []|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|[오늘, 홍조근정훈장을, 받으신...|
|                    |                  []|                  []|
|       사랑하는 경찰관 여러분,|   [사랑하는, 경찰관, 여러분,]|   [사랑하는, 경찰관, 여러분,]|
|                    |                  []|                  []|
|여러분의 헌신적 노력으로 우리의...|[여러분의, 헌신적, 노력으로,...|[여러분의, 헌신적, 노력으로,...|
|                    |                  []|                  []|
|치안의 개선은 국민의 체감으로 ...|[치안의, 개선은, 국민의, 체...|[치안의, 개선은, 국민의, 체...|
|                    |                  []|                  []|
|한국을 찾는 외국 관광객들도 우...|[한국